<a href="https://colab.research.google.com/github/jiya-0905/Mashup-project/blob/main/Mashup_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install pytubefix audio-extract pydub  >/dev/null

In [25]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
from pytubefix import Search
import shutil
import os
import os
from audio_extract import extract_audio
from pydub import AudioSegment
import os
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from google.colab import files
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import re
from google.colab import userdata

In [26]:
def download_videos(singer_name, num_videos):

  results = Search(singer_name)
  for video in results.videos[:num_videos]:
      url=video.watch_url;
      yt = YouTube(url, on_progress_callback = on_progress)
      print(yt.title)
      ys = yt.streams.get_highest_resolution()
      ys.download()

In [27]:
def create_folder():

  # This assumes your videos are in the current working directory
  current_directory = "/content"
  input_folder="/content/inputvideo"
  os.makedirs(input_folder)
  # List all video files
  video_files = [f for f in os.listdir(current_directory) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
  # Move each video file to the input folder
  for video in video_files:

      source = os.path.join(current_directory, video)
      destination = os.path.join(input_folder, video)
      shutil.move(source, destination)
      print(f"Moved {video} to {input_folder}")


In [28]:
def format_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return (f"{minutes:02d}:{remaining_seconds:02d}")

In [29]:
def convert_to_audio(y):

  trim=format_seconds(y)
  input_folder = "/content/inputvideo"  # The folder containing your video files
  output_folder = "/content/outputaudio"  # The folder where audio files will be saved

  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

        # Get all video files
  video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

  for video_file in video_files:
            try:
                # Construct full file paths
                input_path = os.path.join(input_folder, video_file)
                audio_file = os.path.splitext(video_file)[0] + '.mp3'
                output_path = os.path.join(output_folder, audio_file)
                extract_audio(input_path, output_path,start_time=trim)
                print(f'Converted "{video_file}" to "{audio_file}"')
            except Exception as e:
                print(f'Error converting "{video_file}": {str(e)}')


In [30]:
def merge_audio(result):
  audio_folder = "/content/outputaudio"
  audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.mp3')]
  output_file = os.path.join("/content/",result)
  combined = AudioSegment.empty()

  for audio in audio_files:
      # Construct the full file path by joining the audio folder and filename
      audio_path = os.path.join(audio_folder, audio)

      try:
          sound = AudioSegment.from_mp3(audio_path)  # Use the full path
          combined += sound
      except FileNotFoundError:
          print(f"File not found: {audio_path}")
          # Handle the error, e.g., skip the file or raise an exception

  combined.export(output_file, format="mp3")
  print(f"Merged audio saved as: {output_file}")

  for audio in audio_files:
      audio_path = os.path.join(audio_folder, audio)
      try:
          os.remove(audio_path)
      except FileNotFoundError:
          print(f"File not found for deletion: {audio_path}")


In [31]:
def cleanup():
    folders = ['/content/inputvideo', '/content/outputaudio']
    for folder in folders:
        if os.path.exists(folder):
            shutil.rmtree(folder)

In [32]:
def is_valid_email(email):
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None

def send_email(receiver_email, file_path, sender_email, sender_password):
    with progress_output:
        print("\nSending email...")

        # Create message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = "Your Music Mashup is Ready!"

        # Add body
        body = """
        Hello!

        Your music mashup has been created successfully.
        You can find the mashup file attached to this email.

        Enjoy!
        """
        msg.attach(MIMEText(body, 'plain'))

        # Attach file
        with open(file_path, 'rb') as f:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename="{os.path.basename(file_path)}"'
            )
            msg.attach(part)

        # Send email
        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.login(sender_email, sender_password)
            server.send_message(msg)
            server.quit()
            print("Email sent successfully!")
            return True
        except Exception as e:
            print(f"Error sending email: {str(e)}")
            return False


In [33]:
def on_button_click(b):
    create_button.disabled = True
    status_label.value = '<div style="color: #4299e1; margin: 10px 0;">Processing... Please wait.</div>'

    # Validate email
    if not is_valid_email(email_address.value):
        status_label.value = '<div style="color: #f56565; margin: 10px 0;">❌ Please enter a valid email address.</div>'
        create_button.disabled = False
        return
    try:
        with progress_output:
            clear_output()
            if not output_filename.value:
                output_filename.value = 'mashup_result.mp3'

            print("Starting mashup creation process...")
            download_videos(singer_name.value, num_videos.value)
            create_folder()
            convert_to_audio(duration.value)
            merge_audio(output_filename.value)

            # Send email with the mashup
            output_file_path = f"/content/{output_filename.value}"
            email_sent = send_email(
                email_address.value,
                output_file_path,
                userdata.get('gmail_id'),
                userdata.get('hehe')
            )

            if email_sent:
                status_label.value = '<div style="color: #48bb78; margin: 10px 0;">✔️ Mashup created and sent to your email successfully!</div>'
            else:
                status_label.value = '<div style="color: #f56565; margin: 10px 0;">✔️ Mashup created but email sending failed. Downloading file...</div>'
                files.download(output_file_path)

    except Exception as e:
        status_label.value = f'<div style="color: #f56565; margin: 10px 0;">❌ Error: {str(e)}</div>'
    finally:
        cleanup()
        create_button.disabled = False

In [34]:
display(HTML("""
<div style="
    max-width: 800px;
    margin: 20px auto;
    padding: 20px;
    background: white;
    border-radius: 10px;
    box-shadow: 0 0 20px rgba(0,0,0,0.1);
">
    <h1 style="
        text-align: center;
        color: #4a5568;
        margin-bottom: 20px;
        font-family: 'Google Sans', sans-serif;
    ">YouTube Music Mashup Creator</h1>

    <div style="
        padding: 15px;
        background: #f7fafc;
        border-radius: 5px;
        margin-bottom: 20px;
    ">
        <p style="margin: 0; color: #4a5568;">
            Create your own music mashup by entering:
            <ul>
                <li>Singer's name</li>
                <li>Number of videos (more than 10)</li>
                <li>Duration in seconds (more than 20)</li>
            </ul>
        </p>
    </div>
</div>
"""))

# Create form widgets with better styling
style = {'description_width': '150px'}
layout = widgets.Layout(width='750px')

singer_name = widgets.Text(
    description='Singer Name:',
    placeholder='Enter singer name...',
    style=style,
    layout=layout
)

num_videos = widgets.IntSlider(
    description='Number of Videos:',
    min=11,
    max=30,
    step=1,
    value=15,
    style=style,
    layout=layout
)

duration = widgets.IntSlider(
    description='Duration (seconds):',
    min=21,
    max=60,
    step=1,
    value=30,
    style=style,
    layout=layout
)

output_filename = widgets.Text(
    description='Output Filename:',
    placeholder='result.mp3',
    style=style,
    layout=layout
)

progress_output = widgets.Output()
status_label = widgets.HTML(value="")

create_button = widgets.Button(
    description='Create Mashup',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)
email_address = widgets.Text(
    description='Email Address:',
    placeholder='Enter your email...',
    style=style,
    layout=layout
)


display(widgets.VBox([
    singer_name,
    num_videos,
    duration,
    output_filename,
    email_address,
    create_button,
    status_label,
    progress_output
], layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    width='100%',
    padding='20px'
)))

create_button.on_click(on_button_click)